## Imports

In [1]:
# Read stocks
import yfinance as yf

# For DataFrame
import pandas as pd
import numpy as np

## Configuration

In [2]:
# Random companies in the Software-Infrastructure Industry under Technology Sector
SYMBOLS = 'HCP,QLYS,EEFT,ALTR,S,BOX,DOX,PATH,TDC,SPLK,INFA,IOT,CFLT,GTLB,ACIW,MDB,CRWD,GEN'.split(',')
# A dictionary to hold scores for each symbol
symbol_to_score = {}

## Methods to calculate the Z-Score

In [3]:
# ratio_x_1: working capital / total assets
def ratio_x_1(ticker:yf.ticker.Ticker) -> float:
    df = ticker.balance_sheet
    working_capital = df.loc['Current Assets'].iloc[0] - df.loc['Current Liabilities'].iloc[0]
    total_assets = df.loc['Total Assets'].iloc[0]
    return working_capital/total_assets

# retained earnings / total assets
def ratio_x_2(ticker:yf.ticker.Ticker) -> float:
    df = ticker.balance_sheet
    retained_earnings = df.loc['Retained Earnings'].iloc[0]
    total_assets = df.loc['Total Assets'].iloc[0]
    return retained_earnings/total_assets

# earnings before interest and tax / total assets
def ratio_x_3(ticker:yf.ticker.Ticker) -> float:
    df = ticker.income_stmt
    ebit = df.loc['EBIT'].iloc[0]
    total_assets = ticker.balance_sheet.loc['Total Assets'].iloc[0]
    return ebit/total_assets

# market value of equity / total liabilities
def ratio_x_4(ticker:yf.ticker.Ticker) -> float:
    equity_market_value = ticker.info['sharesOutstanding'] * ticker.info['currentPrice']
    total_liabilities = ticker.balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
    return equity_market_value/total_liabilities

# sales / total assets
def ratio_x_5(ticker:yf.ticker.Ticker) -> float:
    df = ticker.income_stmt
    sales = df.loc['Total Revenue'].iloc[0]
    total_assets = ticker.balance_sheet.loc['Total Assets'].iloc[0]
    return sales/total_assets

## Calculate Z-Score

In [4]:
def z_score(ticker:yf.ticker.Ticker) -> float:
    ratio_1 = ratio_x_1(ticker)
    ratio_2 = ratio_x_2(ticker)
    ratio_3 = ratio_x_3(ticker)
    ratio_4 = ratio_x_4(ticker)
    ratio_5 = ratio_x_5(ticker)
    # Z = 1.2X1 + 1.4X2 + 3.3X3 + 0.6X4 + 1.0X5.
    zscore = 1.2*ratio_1 + 1.4*ratio_2 + 3.3*ratio_3 + 0.6*ratio_4 + 1.0*ratio_5
    return zscore

In [5]:
# Loop through symbol list
for symbol in SYMBOLS:
    ticker = yf.Ticker(symbol)
    symbol_to_score[symbol]=z_score(ticker)
symbol_to_score

{'HCP': 3.966377993725474,
 'QLYS': 9.440156406216962,
 'EEFT': 2.2088606360107548,
 'ALTR': 4.889793303042627,
 'S': 5.550169753018658,
 'BOX': 0.8240135124924809,
 'DOX': 4.883301178383062,
 'PATH': 7.845345253857153,
 'TDC': 1.0667425910691386,
 'SPLK': 2.2338104274185366,
 'INFA': 1.7443872763487027,
 'IOT': 4.755161281413089,
 'CFLT': 2.287857500338857,
 'GTLB': 11.211532361994733,
 'ACIW': 2.2943724120376174,
 'MDB': 9.53030235538038,
 'CRWD': 11.829298257627572,
 'GEN': 0.8740704803810675}

## Adds styles to DF

In [6]:
def highlight_distress(val):
    return 'background-color: indianred' if val != '' else ""
    
def highlight_grey(val):
    return 'background-color: grey' if val != '' else ""

def highlight_safe(val):
    return 'background-color: green' if val != '' else ""

def format_score(val):
    try:
        return '{:.2f}'.format(float(val))
    except:
        return ''
    
def make_pretty(styler):
    # No index
    styler.hide(axis='index')
    
    # Column formatting
    styler.format(format_score, subset=['Distress Zone', 'Grey Zone', 'Safe Zone'])

    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Distress Zone', 'Grey Zone', 'Safe Zone'], **{'text-align': 'center', 'width': '100px'})

    styler.map(highlight_grey, subset=['Grey Zone'])
    styler.map(highlight_safe, subset=['Safe Zone'])
    styler.map(highlight_distress, subset=['Distress Zone'])
    return styler

## Display DF

In [7]:
distress = [''] * len(SYMBOLS)
grey = [''] * len(SYMBOLS)
safe = [''] * len(SYMBOLS)

for idx, zscore in enumerate(symbol_to_score.values()):
    if zscore <= 1.8:
        distress[idx] = zscore
    elif zscore > 1.8 and zscore <= 2.99:
        grey[idx] = zscore
    else:
        safe[idx] = zscore

# Create a dictionary for the DF
data_dict = {'Symbol': SYMBOLS, 'Distress Zone': distress, 'Grey Zone': grey, 'Safe Zone': safe} 
df = pd.DataFrame.from_dict(data_dict)
# Drop any rows with NaN values
df.dropna(inplace=True)

styles = [
    dict(selector='td', props=[('font-size', '10pt'),('border-style','solid'),('border-width','1px')]),
    dict(selector='th.col_heading', props=[('font-size', '11pt'),('text-align', 'center')]),
    dict(selector='caption', props=[('text-align', 'center'),
                                     ('font-size', '14pt'), ('font-weight', 'bold')])
]
# Apply styles
df_styled = df.style.set_table_styles(styles)
df.style.pipe(make_pretty).set_caption('Altman Z Score').set_table_styles(styles)

Symbol,Distress Zone,Grey Zone,Safe Zone
HCP,,,3.97
QLYS,,,9.44
EEFT,,2.21,
ALTR,,,4.89
S,,,5.55
BOX,0.82,,
DOX,,,4.88
PATH,,,7.85
TDC,1.07,,
SPLK,,2.23,
